In [3]:
import time
import numpy as np
import matplotlib.pyplot as plt
from ParticleTracer import ParticleTracer
from particleTracerLattice import ParticleTracerLattice

In [5]:
lattice = ParticleTracerLattice(200.0)
fileBend1='benderSeg1.txt'
fileBend2 = 'benderSeg2.txt'
fileBender1Fringe='benderFringeCap1.txt'
fileBenderInternalFringe1='benderFringeInternal1.txt'
fileBender2Fringe='benderFringeCap2.txt'
fileBenderInternalFringe2='benderFringeInternal2.txt'
file2DLens='lens2D.txt'
file3DLens='lens3D.txt'
fileCombiner='combinerData.txt'
yokeWidth=.0254*5/8
numMagnets=110
extraSpace=1e-3
Lm=.0254
rp=.0125
Llens1=.3
rb=1.0

lattice.add_Lens_Sim_With_Caps(file2DLens, file3DLens, Llens1)
lattice.add_Combiner_Sim(fileCombiner)
lattice.add_Lens_Sim_With_Caps(file2DLens, file3DLens, Llens1)
lattice.add_Bender_Sim_Segmented_With_End_Cap(fileBend1,fileBender1Fringe,fileBenderInternalFringe1,Lm,None,rb,extraSpace,yokeWidth)
lattice.add_Lens_Sim_With_Caps(file2DLens, file3DLens, None)
lattice.add_Bender_Sim_Segmented_With_End_Cap(fileBend2,fileBender2Fringe, fileBenderInternalFringe2, Lm, None, rb, extraSpace,yokeWidth)
lattice.end_Lattice()
particleTracer=ParticleTracer(lattice)
#lattice.show_Lattice()

In [ ]:
lattice.elList[2].forceFact = 0.11379310
lattice.elList[4].forceFact = 0.18275862

In [17]:
vi=np.asarray([-200.0,0,0])
qi=np.asarray([-1e-10,0,0])
particleTracer.trace(qi,vi,1e-5,10*lattice.totalLength/200.0,fastMode=True)


(31.82476177406698, True, 1)